In [ ]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
#get_ipython().magic(u'matplotlib inline')
import cPickle
from multiprocessing import Pool
from sklearn import preprocessing

In [ ]:
def dataframe2corpus(df,name_file):
    '''
    df : dataframe containing the data
    name_file : name of file saved containing the dictionnary for the lda model
    '''
    non_num_columns=['AffinityCodeId','SelectedPackage','NameOfPolicyProduct','SCID']
    le = preprocessing.LabelEncoder()
    for column in non_num_columns:
        df[column]=le.fit_transform(df[column].as_matrix())
    dropped_columns=['CustomerMD5Key','ReceivedDateTime','Unnamed: 0','TodayDate']
    all_columns=df.columns.tolist()
    for column_name in dropped_columns:
        all_columns.remove(column_name)

    reduced_df=df[all_columns]
    
    # List of column names with categorical variables
    categ_vars=[]
    for column in reduced_df.columns:
        if len(np.unique(reduced_df[column]))<50:    # we don't keep columns with too many different values
            categ_vars+=[column]

# Encode the data as a bag of words to feed gensim LDA
    equiv_texts=[]
    all_cat=reduced_df[categ_vars].as_matrix().tolist()
    for row in all_cat:
        text_row=[str(categ_vars[i])+':'+str(word) for (i,word) in enumerate(row)]
        equiv_texts+=[text_row]
    
    
    #dictionary = corpora.Dictionary(equiv_texts)
    #dictionary.save('topics_'+str(name_file)+'.dict') # dictionnary id2word
    dictionary=corpora.Dictionary.load('topics_alex.dict')
    
    
    
    corpus = [dictionary.doc2bow(text) for text in equiv_texts]
    corpora.MmCorpus.serialize('topics_'+str(name_file)+'.mm', corpus) # serialized dictionnary (for fast lda)
    return



In [ ]:


n_topics=5

name_file='final'

In [ ]:
X_test=pd.read_csv('X_test.csv')
#X_train=pd.read_csv('X_train.csv')

df=X_test
del X_test

dataframe2corpus(df,name_file)
lenX=np.shape(df.as_matrix())[0]
del df



In [ ]:
# step 2 : 

lda=cPickle.load(open('lda_alex_'+str(n_topics)+'_topics.p','rb')) # load lda model trained on the big corpus

mm_test = corpora.MmCorpus('topics_'+str(name_file)+'.mm')
id2word = corpora.Dictionary.load('topics_alex.dict')
   
ldamm=lda[mm_test]
del mm_test,lda # saving memory
L_indices=list(range(lenX)) ##########


def transform_vocab(idx_doc_bow):
    global n_topics,ldamm
    bow_converted=ldamm[idx_doc_bow]
    local_array=[0]*(n_topics+1)
    for feat_tuple in bow_converted:
        local_array[feat_tuple[0]+1]=feat_tuple[1]
    return local_array
        
# Compute the feature vector :
p=Pool(10)
a=p.map(transform_vocab,L_indices)

cPickle.dump(a,open('sauvegarde_qui_sauve_la_vie.p','wb'))
topic_features_df=pd.DataFrame(a)
del a
topic_features_df=topic_features_df[topic_features_df.columns[1:]]
topic_features_df.columns=['lda_feature_'+str(i) for i in range(len(topic_features_df.columns))]
topic_features_df.to_csv('lda_features_'+str(n_topics)+'_'+str(name_file)+'_topics_df.csv',index=None)
#compute_lda_feature(n_topics,name_file)


In [3]:
n_topics=5
lda=cPickle.load(open('lda_alex_'+str(n_topics)+'_topics.p','rb')) # load lda model trained on the big corpus

In [6]:
lda.print_topic(3)

u'0.053*SelectedPackage:1.0 + 0.052*FirstDriverDrivingLicenceType:1.0 + 0.051*IsPolicyholderAHomeowner:1.0 + 0.050*FirstDriverMaritalStatus:2.0 + 0.049*AllDriversNbConvictions:0.0 + 0.045*RatedDriverNumber:1.0 + 0.039*CarParkingTypeId:2.0 + 0.038*PolicyHolderNoClaimDiscountYears:9.0 + 0.038*CoverIsNoClaimDiscountSelected:1.0 + 0.036*CarDrivingEntitlement:2.0'